In [32]:
import requests
from bs4 import BeautifulSoup
import re
from dataclasses import dataclass, field
from enum import Enum
import datetime as dt
from pprint import pprint

In [33]:
class TipoClase(Enum):
    EXPOSITIVA = 'CLE'
    INTERACTIVA = 'CLIL'
    SEMINARIO = 'TODO'

class DiaSemana(Enum):
    LUNES = 'Lunes'
    MARTES = 'Martes'
    MIERCOLES = 'Miércoles'
    JUEVES = 'Jueves'
    VIERNES = 'Viernes'

@dataclass
class HoraClase:
    grupo: int
    dia_semana: DiaSemana
    hora_inicio: dt.time
    duracion: dt.timedelta
    aula: str
    tipo: TipoClase

@dataclass
class Examen:
    fecha: dt.datetime
    duracion: dt.timedelta
    aula: set[str]

@dataclass
class Clase:
    horario: list[HoraClase] = field(default_factory=list)
    examenes: list[Examen] = field(default_factory=list)

In [41]:
def datos_clase(url):
	r = requests.get(url)
	soup = BeautifulSoup(r.text, 'html.parser')

	clase = Clase()
	dia = DiaSemana.LUNES

	tb_cl = soup.find('caption', text=re.compile('semestre')).parent
	tb_ex = soup.find('caption', text=re.compile('Exámenes')).parent

	for c in tb_cl.findAll('tr'):
		if c.find('th'):
			dia = DiaSemana(c.find('th').text)
			continue

		tipo, grupo = c.find_all('td')[1].text.split(' ')[-1][1:].split('_')
		hora_inicio, hora_fin = list(map(lambda x: dt.datetime.strptime(x, '%H:%M'), c.find_all('td')[0].text.split('-')))

		clase.horario.append(HoraClase (
			grupo = int(grupo),
			dia_semana = dia,
			hora_inicio = hora_inicio.time(),
			duracion = hora_fin - hora_inicio,
			aula = c.find_all('td')[2].text.split(' ')[-1],
			tipo = TipoClase(tipo)
		))

	for e in tb_ex.findAll('tr', class_='target-items-selector'):
		str_fecha, str_hora_fin = e.find_all('td')[0].text.split('-')
		fecha = dt.datetime.strptime(str_fecha, '%d.%m.%Y %H:%M')
		hora_fin = dt.datetime.combine(fecha.date(), dt.datetime.strptime(str_hora_fin, '%H:%M').time())
		aula = e.find_all('td')[2].text.split(' ')[-1]

		examen = next(filter(lambda e: e.fecha == fecha, clase.examenes), None)
		if examen is None:
			clase.examenes.append(Examen(
				fecha = fecha,
				duracion = hora_fin - fecha,
				aula = { aula }
			))
			continue
		examen.aula.add(aula)

	return clase

In [42]:
c = datos_clase('https://www.usc.gal/es/centro/escuela-tecnica-superior-ingenieria/horarios/materias/programacion-orientada-objetos-17632-16883-2-98046')
pprint(c)

Clase(horario=[HoraClase(grupo=3,
                         dia_semana=<DiaSemana.LUNES: 'Lunes'>,
                         hora_inicio=datetime.time(11, 30),
                         duracion=datetime.timedelta(seconds=9000),
                         aula='I5',
                         tipo=<TipoClase.INTERACTIVA: 'CLIL'>),
               HoraClase(grupo=4,
                         dia_semana=<DiaSemana.MARTES: 'Martes'>,
                         hora_inicio=datetime.time(11, 30),
                         duracion=datetime.timedelta(seconds=9000),
                         aula='I8',
                         tipo=<TipoClase.INTERACTIVA: 'CLIL'>),
               HoraClase(grupo=1,
                         dia_semana=<DiaSemana.MARTES: 'Martes'>,
                         hora_inicio=datetime.time(15, 0),
                         duracion=datetime.timedelta(seconds=7200),
                         aula='A2',
                         tipo=<TipoClase.EXPOSITIVA: 'CLE'>),
               HoraCl